In [23]:
import geopandas as gpd

In [24]:
# city = "Providence"
city = "Cambridge"
addresses = gpd.read_file(f"../data/{city}/filtered_{city}_addresses.geojson")
buildings = gpd.read_file(f"../data/{city}/filtered_{city}_buildings.geojson")
places = gpd.read_file(f"../data/{city}/filtered_{city}_places.geojson")
building_parts = gpd.read_file(f"../data/{city}/filtered_{city}_buildingparts.geojson")

In [28]:
merged = building_parts.merge(
    buildings,
    left_on="building_id",
    right_on="id",
    suffixes=("_part", "_building")
)

# Select and rename columns to include desired attributes
result = merged[[
    'id_part',
    'id_building',
    # From buildings
    'subtype',
    'class', 
    'names_building', 
    'level_building', 
    'height_building', 
    'num_floors_building',
    'geometry_building', 
    
    # From building_parts
    'names_part', 
    'height_part', 
    'num_floors_part', 
    'geometry_part'
]]

# Rename columns for clarity
result = result.rename(columns={
    'id_building': 'building_id',
    'id_part': 'part_id',
    'names_building': 'building_names',
    'height_building': 'building_height',
    'num_floors_building': 'building_num_floors',
    'geometry_building': 'building_geometry',
    'names_part': 'part_names',
    'height_part': 'part_height',
    'num_floors_part': 'part_num_floors',
    'geometry_part': 'part_geometry'
})
unassigned_building_parts = building_parts[~building_parts.id.isin(result.part_id)]
result.head(2)
# unassigned_building_parts.head(2)

,part_id,building_id,subtype,class,building_names,level_building,building_height,building_num_floors,building_geometry,part_names,part_height,part_num_floors,part_geometry
0,08b194e535cc3fff0201c86819aa93a6,08b194e535cc0fff0200f3bafffdd30e,None,None,None,1.0,NaN,NaN,"POLYGON ((0.13065 52.17429, 0.13096 52.17385, ...","{ ""primary"": null, ""common"": null, ""rules"": nu...",NaN,1.0,"POLYGON ((0.13077 52.17411, 0.13096 52.17385, ..."
1,08b194e535cc6fff0201d6e6d5305852,08b194e535cc0fff0200f3bafffdd30e,None,None,None,1.0,NaN,NaN,"POLYGON ((0.13065 52.17429, 0.13096 52.17385, ...","{ ""primary"": null, ""common"": null, ""rules"": nu...",NaN,2.0,"POLYGON ((0.13065 52.17429, 0.13077 52.17411, ..."


In [29]:
import pandas as pd

result = result.rename(columns={"building_geometry": "geometry"})
places = places.to_crs(result.crs)
result = result.rename(columns={"geometry": "building_geometry"})
places_with_buildings = gpd.sjoin(
    places, 
    result[['building_id', 'building_geometry']].rename(columns={'building_geometry': 'geometry'}),
    how='inner', 
    predicate='intersects'
)

places_with_parts = gpd.sjoin(
    places, 
    result[['part_id', 'part_geometry']].rename(columns={'part_geometry': 'geometry'}),
    how='inner', 
    predicate='intersects'
)

places_with_geometries = pd.concat([places_with_buildings, places_with_parts])

places_with_geometries = places_with_geometries[[
    'id', 'names', 'categories', 'confidence', 'brand', 'index_right'
]].rename(columns={
    'id': 'places_id',
    'names': 'places_names',
    'categories': 'places_categories',
    'confidence': 'places_confidence',
    'brand': 'places_brand',
    'index_right': 'result_index'
})
if 'level_0' in result.columns:
    result = result.drop(columns=['level_0'])
result = result.reset_index().merge(
    places_with_geometries, 
    left_index=True, 
    right_on='result_index', 
    how='left'
)

result = result.drop(columns=['result_index'])
unassigned_places = places[~places.id.isin(places_with_geometries.places_id)]
# result.head(2)
unassigned_places.head(2)


,id,version,sources,names,categories,confidence,addresses,brand,geometry
0,08f194e53004498a03714439e04ee28b,0,"[ { ""property"": """", ""dataset"": ""msft"", ""record...","{ ""primary"": ""Prestige External Cleaning"", ""co...","{ ""primary"": ""road_contractor"", ""alternate"": [...",0.770000,"[ { ""freeform"": ""8 Beech Drive, Trumpington"", ...",None,POINT (0.10862 52.16474)
1,08f194e53014b60c03855fe67dca7331,0,"[ { ""property"": """", ""dataset"": ""meta"", ""record...","{ ""primary"": ""Chameleon Decorators & Windows R...","{ ""primary"": ""painting"", ""alternate"": [ ""contr...",0.988058,"[ { ""freeform"": ""3 Harness Cl"", ""locality"": nu...",None,POINT (0.1088 52.16629)


In [30]:
result = result.rename(columns={"part_geometry": "geometry"})
addresses = addresses.to_crs(result.crs)
result = result.rename(columns={"geometry": "part_geometry"})
print(addresses.columns)
addresses_with_buildings = gpd.sjoin(
    addresses, 
    result[['building_id', 'building_geometry']].rename(columns={'building_geometry': 'geometry'}),
    how='inner', 
    predicate='intersects'
)
addresses_with_parts = gpd.sjoin(
    addresses, 
    result[['part_id', 'part_geometry']].rename(columns={'part_geometry': 'geometry'}),
    how='inner', 
    predicate='intersects'
)
addresses_with_geometries = pd.concat([addresses_with_buildings, addresses_with_parts])

addresses_with_geometries = addresses_with_geometries[[
    'id', 'postcode', 'street', 'number', 'unit', 'address_levels', 'index_right'
]].rename(columns={
    'id': 'address_id',
    'postcode': 'address_postcode',
    'street': 'address_street',
    'number': 'address_number',
    'unit': 'address_unit',
    'address_levels': 'address_levels',
    'index_right': 'result_index'
})

if 'level_0' in result.columns: # not sure why i need to use this
    result = result.drop(columns=['level_0'])

result = result.reset_index().merge(
    addresses_with_geometries, 
    left_index=True, 
    right_on='result_index', 
    how='left'
)

result = result.drop(columns=['result_index'])
unassigned_addresses = addresses[~addresses.id.isin(addresses_with_geometries.address_id)]

# result.head(2)
unassigned_addresses.head(2)

Index(['geometry'], dtype='object')


KeyError: "['id', 'postcode', 'street', 'number', 'unit', 'address_levels'] not in index"

In [31]:
if 'level_0' in result.columns:
    result = result.drop(columns=['level_0'])
result["part_wkt"] = result["part_geometry"].to_wkt()
result.drop(columns=["part_geometry"], inplace=True)

result.to_file(f"../data/Providence/{city}_merged.geojson", driver='GeoJSON')
unassigned_building_parts.to_file(f"../data/Providence/{city}_unassigned_building_parts.geojson", driver='GeoJSON')
unassigned_places.to_file(f"../data/Providence/{city}_unassigned_places.geojson", driver='GeoJSON')
unassigned_addresses.to_file(f"../data/Providence/{city}_unassigned_addresses.geojson", driver='GeoJSON')